### Transform the Refunda data 
- 1.Extract specfic portion of the string from refund_reason uisng the split function
- 2.Extract specific portion of the string from refund_reason using regexp_extract function
- 3.Extract date and timie from the refund_timestamp
- 4.Write transformed data into the silver schema

In [0]:
df = spark.table('gizmobox.bronze.refunds')

In [0]:
display(df)

1.Extract specfic portion of the string from refund_reason uisng the split function

In [0]:
from pyspark.sql import functions as F

df_split_refunds =(
    df.select(
        'refund_id',
        'payment_id',
        'refund_timestamp',
        'refund_amount',
        F.split("refund_reason",":")[0].alias("refund_reason"),
        F.split("refund_reason",":")[1].alias("refund_source")
    )
)
display(df_split_refunds)

2.Extract specific portion of the string from refund_reason using regexp_extract function

In [0]:
df_split_refunds_extract = (
    df.select(
        "refund_id",
        "payment_id",
        'refund_timestamp',
        "refund_amount",
        F.regexp_extract("refund_reason",'^([^:]+):',1).alias("refund_reason"),
        F.regexp_extract("refund_reason",'^[^:]+:(.*)$',1).alias("refund_source")
    )
    )
display(df_split_refunds_extract)

In [0]:
df_transformed_refunds = (
    df_split_refunds_extract.select(
        "refund_id",
        "payment_id",
        F.date_format("refund_timestamp",'yyyy-MM-dd').cast("date").alias("refund_date"),
        F.date_format("refund_timestamp",'HH:mm:ss').alias("refund_time"),
        "refund_amount",
        "refund_reason",
        "refund_source")
    )
display(df_transformed_refunds)


4.Write transformed data into the silver schema

In [0]:
df_transformed_refunds.writeTo("gizmobox.silver.py_refunds").createOrReplace()

In [0]:
%sql
select * from gizmobox.silver.py_refunds;